In [28]:
import shopify
import datetime
from datetime import date, timedelta, time, datetime
import pandas as pd
from process_data.process_adress import process_adress
import os
import unidecode
import sys

droppedFile = sys.argv[1]

SINCE_NUM_DAYS = 3
UNTIL_NUM_DAYS = 0

LAPOSTE_COLUMNS = ['id', 'Nom', 'Prénom']

API_KEY_TRACK_ORDERS = os.environ.get('API_KEY_TRACK_ORDERS')
API_PASSWORD_TRACK_ORDERS = os.environ.get('API_PASSWORD_TRACK_ORDERS')
SHOP_NAME = 'iriscosmetics'
shop_url = "https://%s.myshopify.com/admin" % (SHOP_NAME)

shopify.ShopifyResource.set_user(API_KEY_TRACK_ORDERS)
shopify.ShopifyResource.set_password(API_PASSWORD_TRACK_ORDERS)
shopify.ShopifyResource.set_site(shop_url)
#The following code prints the amount of orders up in your shopify store in command prompt window
#shop = shopify.Shop.current()
def date_from_today(n):
    date_today = date.today()
    return str(datetime.combine(date_today, time.min) - timedelta(days = n, hours = 2))

def process_laposte_track(path):
    tracking_number_df = pd.read_csv(path, encoding = "ISO-8859-1", sep =';')
    tracking_number_df['id'] = (tracking_number_df['Prénom'] + tracking_number_df['Nom']).apply(lambda x: unidecode.unidecode(x).replace(' ', '').lower())
    return tracking_number_df[['id', 'Numéro de suivi']]

def shopify_orders_df(orders):
    laposte_df = pd.DataFrame(columns = LAPOSTE_COLUMNS)
    for order in orders:
        dict_order = order.to_dict()
        shipping_data = dict_order['shipping_address']
        df = pd.DataFrame(columns = LAPOSTE_COLUMNS)
        df['order_id'] = [str(dict_order['id'])]
        df['Nom'] = [shipping_data['last_name'].replace('  ', ' ').strip()]
        df['Prénom'] = shipping_data['first_name'].replace('  ', ' ').strip()       
        laposte_df = pd.concat([laposte_df, df])
        laposte_df = laposte_df.reset_index(drop = True)
    laposte_df['id'] = (laposte_df['Prénom'] + laposte_df['Nom']).apply(lambda x: unidecode.unidecode(x).replace(' ', '').lower())
    return laposte_df

yesterday = str(date.today() - timedelta(1))
orders = shopify.Order.find(created_at_min = date_from_today(SINCE_NUM_DAYS),
                           created_at_max = date_from_today(UNTIL_NUM_DAYS))
df_orders = shopify_orders_df(orders)


C:\Users\edmon\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [29]:
path = 'suivi_prepa_facile/LaPoste_Export_suivis_envois_11-06-2019.csv'
tracking_number_df = process_laposte_track(path)

In [32]:
len(df_orders)

39

In [31]:
pd.merge(tracking_number_df, df_orders, on = 'id')

,id,Numéro de suivi,Nom,Prénom,order_id
0,chriswassbo,2L 006 124 9531 7,WASSBO,Chris,1159018086478
1,catherinevergnas,2L 006 124 9530 0,Vergnas,Catherine,1160003682382
2,florencesegard,2L 006 124 9529 4,Segard,Florence,1159588347982
3,corinnerosellon,2L 006 124 9528 7,Rosellon,Corinne,1158926991438
4,daniellerazigade,2L 006 124 9527 0,Razigade,Danielle,1159483916366
5,solangeravassod,2L 006 124 9526 3,RAVASSOD,Solange,1158900023374
6,michatpolloni,2L 006 124 9525 6,POLLONI,MICHAT,1159205519438
7,tassaditmessani,2L 006 124 9524 9,Messani,Tassadit,1159601455182
8,sophiemartagon,2L 006 124 9523 2,Martagon,sophie,1160016986190
9,jacquelinemaguet,2L 006 124 9521 8,maguet,jacqueline,1158545211470


In [22]:
shopify_order = shopify.Order.find(1160818557006)
new_fulfillment = shopify.Fulfillment({ 'order_id': shopify_order.id, 'line_items': shopify_order.line_items })
new_fulfillment.tracking_company = 'La Poste'
new_fulfillment.tracking_numbers = ['2L00612495225']
new_fulfillment.save()

False

In [27]:
orders[0].to_dict()['created_at']

'2019-06-09T00:04:18+02:00'

In [29]:
date.today()

datetime.date(2019, 6, 9)

In [37]:
datetime.strptime('2019-06-09T00:04:18+02:00')

SyntaxError: EOL while scanning string literal (<ipython-input-37-090332da7a5e>, line 1)

In [50]:
df_orders['Nom'][6] = 'Maelys Amedee'

In [66]:
df_orders.to_csv(subdir + '/' + yesterday +'.csv', sep = ';')

In [64]:
import random
df_orders['Poids'] = str(random.randint(20,100))
df_orders['Poids'] = df_orders['Poids'].apply(lambda x: str(random.randint(20,100)))

In [65]:
df_orders['Poids']

1      26
2      41
3      36
4     100
5      25
6      26
7      91
8      72
9      43
10     63
11     89
12     27
13     74
14     35
15     81
16     87
17     95
18     77
19     83
Name: Poids, dtype: object

In [67]:
'MSTest, JUnit, TestNG, NUni, DUnit,
PyUnit, Ruby, PHPUnit, TestComplete.split(' ')

['',
 'MSTest',
 'JUnit',
 'TestNG',
 'NUni',
 'DUnit',
 'PyUnit',
 'Ruby',
 'PHPUnit',
 'TestComplete']

In [44]:
import pandas as pd
import unidecode
path = 'suivi_prepa_facile/LaPoste_Export_suivis_envois_09-06-2019.csv'
tracking_number_df = pd.read_csv(path, encoding = "ISO-8859-1", sep =';')
tracking_number_df['id'] = (tracking_number_df['Prénom'] + tracking_number_df['Nom']).apply(lambda x: unidecode.unidecode(x).replace(' ', '').lower())

In [45]:
tracking_number_df['id']

0              anniestaquet
1     frederiqueroigborotra
2             lilyanerogeau
3       etiennettemichalski
4        maelysmaelysamedee
5          claudinelemagnen
6         muriellechevalier
7               amalladghem
8         mireillekauffmann
9     mariechristineimbault
10          stephaniehenric
11         teresaguincestre
12     genevievegaudilliere
13       stephaniegatticchi
14       dominiquechiarelli
15            ginettecastra
16             josianebucci
17             sandrineamil
18              cynthiaabel
Name: id, dtype: object

In [39]:
df['id_'] = (df['Prénom']+df['Nom']).apply(lambda x: unidecode.unidecode(x).replace(' ', '').lower())

In [40]:
df['id_']

0              anniestaquet
1     frederiqueroigborotra
2             lilyanerogeau
3       etiennettemichalski
4        maelysmaelysamedee
5          claudinelemagnen
6         muriellechevalier
7               amalladghem
8         mireillekauffmann
9     mariechristineimbault
10          stephaniehenric
11         teresaguincestre
12     genevievegaudilliere
13       stephaniegatticchi
14       dominiquechiarelli
15            ginettecastra
16             josianebucci
17             sandrineamil
18              cynthiaabel
Name: id_, dtype: object

In [ ]:
c = a*b

if c>clim:
    
else:
    